In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


In [10]:
x_df, y_df = prep.shuffle_data(joined_df, rand, seed=seed)

In [11]:
def objective(h_params, X, y):
    print(h_params)
    model = GradientBoostingRegressor(
        loss=h_params[0],
        learning_rate=h_params[1],
        n_estimators=h_params[2],
        max_depth=h_params[3],
        max_features=h_params[4],
        alpha=h_params[5]
    )
    # Standardize all variables (except one-hots)
    # Don't really need to do this for decision tree methods, but might as well
    # repeat the procedure that we're doing for the SVMs, just for posterity
    pipeline = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), ["age_at_diagnosis"] + list(norm_filtered_matrisome_counts_t_df.columns[1:]))
        ], remainder="passthrough"),
        model
    )
    ttr = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    return -np.mean(cross_val_score(
        ttr,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="neg_mean_absolute_error"
    ))

In [12]:
space = [
    # We skip "ls" since we're optimizing for mean absolute error in the h_param opt.
    Categorical(["lad", "huber", "quantile"], name="loss"),
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Real(0.5, 0.9, name="alpha"),
]
n_initial = 10 * (len(space[0].categories) + len(space[4].categories))
n_calls = 50 * (len(space[0].categories) + len(space[4].categories))

In [13]:
res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, "opt_gbr_h_params_v2.tsv", sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
['quantile', 0.04860832066690728, 818, 3, 'log2', 0.5039029398683332]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.7642
Function value obtained: 574.6946
Current minimum: 574.6946
Iteration No: 2 started. Evaluating function at random point.
['lad', 0.06923792863971574, 882, 3, 'log2', 0.6073125230846899]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.3184
Function value obtained: 588.0534
Current minimum: 574.6946
Iteration No: 3 started. Evaluating function at random point.
['quantile', 0.027780951796428976, 577, 3, 'log2', 0.8564436383356643]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.0440
Function value obtained: 653.9022
Current minimum: 574.6946
Iteration No: 4 started. Evaluating function at random point.
['huber', 0.04993730401749343, 291, 4, 'auto', 0.8676227203709375]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 3.8500

KeyboardInterrupt: 